# 第2章: グラフの制御フロー

## 準備

以下のセルを順番に実行して、演習に必要な環境をセットアップします。

### ライブラリのインストール

このセルは、LangGraphおよび関連するLangChainライブラリをインストールします。実行には数分かかる場合があります。
ご利用になるLLMプロバイダーに応じて、コメントアウトを解除して必要なライブラリをインストールしてください。

In [ ]:
# === ライブラリのインストール ===
# 基本ライブラリ (LangGraphとLangChain Core)
!pip install -qU langchain langgraph

# --- LLMプロバイダー別ライブラリ ---
# OpenAI (GPTシリーズ)
# !pip install -qU langchain_openai

# Azure OpenAI
# !pip install -qU langchain_openai

# Google Cloud Vertex AI (Gemini, PaLM等)
# !pip install -qU langchain_google_vertexai

# Google Gemini API
# !pip install -qU langchain_google_genai

# Anthropic (Claudeシリーズ)
# !pip install -qU langchain_anthropic

# Amazon Bedrock
# !pip install -qU langchain_aws boto3

# --- その他の推奨ライブラリ ---
!pip install -qU python-dotenv pygraphviz pydotplus graphviz

### OpenAI APIキーの設定

このノートブックでは、いくつかの問題でOpenAIのLLMを使用します。
事前にOpenAIのAPIキーを取得し、環境変数 `OPENAI_API_KEY` に設定するか、Google Colabの場合はColabのシークレットマネージャーに `OPENAI_API_KEY` という名前でキーを登録してください。

`.env` ファイルを使用する場合:
1. リポジトリのルートにある `.env.sample` をコピーして `.env` という名前のファイルを作成します。
2. `.env` ファイルを開き、`OPENAI_API_KEY="sk-..."` のようにご自身のAPIキーを記述して保存します。

以下のセルは、設定されたAPIキーをロードします。

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

try:
    from google.colab import userdata
    if "OPENAI_API_KEY" in userdata.get_all():
        os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    print("OpenAI APIキーをColabシークレットからロードしました。")
except ImportError:
    if os.getenv("OPENAI_API_KEY"):
        print("OpenAI APIキーを.envファイルからロードしました。")
    else:
        print("OpenAI APIキーが見つかりません。環境変数に設定するか、Colabシークレットに登録してください。")

# LLMの初期化 (この章では主にOpenAIのモデルを使用する想定)
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print(f"LLM ({llm.model_name}) の準備ができました。")

---

## 問題001: 条件付きエッジによる分岐の導入

### 課題
LangGraphの強力な機能の一つである条件付きエッジを導入し、グラフの実行パスを動的に制御する方法を学びましょう。ここでは、入力された数値が偶数か奇数かによって、異なる処理を行うグラフを作成します。

*   **学習内容:** `add_conditional_edges`を使用して、グラフの実行パスを動的に制御する方法を学びます。ルーター関数がどのように次のノードを決定するのか、そして状態が分岐間でどのように共有されるかを理解します。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class ConditionalGraphState(TypedDict):
    messages: Annotated[list, add_messages]
    number: int # 数値を保持する状態

# --- ノード定義 (Nodes) ---
def get_number_node(state: ConditionalGraphState):
    last_message = state["messages"][-1].content
    try:
        num = int(last_message)
        return {"number": num}
    except ValueError:
        return {"number": 0, "messages": [AIMessage(content="無効な数値入力です。0として処理します。")]}

def even_node(state: ConditionalGraphState):
    return {"messages": [AIMessage(content=f"{state['number']} は偶数です。")]}

def odd_node(state: ConditionalGraphState):
    return {"messages": [AIMessage(content=f"{state['number']} は奇数です。")]}

# --- ルーター関数 ---
def number_router(state: ConditionalGraphState):
    if state["number"] % 2 == 0:
        return "even_branch"
    else:
        return "odd_branch"

# --- グラフ構築 ---
workflow = StateGraph(ConditionalGraphState)
workflow.add_node("input_node", get_number_node)
workflow.add_node("even_processor", even_node)
workflow.add_node("odd_processor", odd_node)

workflow.set_entry_point("input_node")
workflow.add_conditional_edges(
    "input_node",
    number_router,
    {
        "even_branch": "even_processor",
        "odd_branch": "odd_processor"
    }
)
workflow.add_edge("even_processor", END)
workflow.add_edge("odd_processor", END)

graph = workflow.compile()

# --- 実行 --- 
print("--- 偶数のテスト ---")
inputs_even = {"messages": [HumanMessage(content="42")]}
final_state_even = graph.invoke(inputs_even)
print(f"最終応答: {final_state_even['messages'][-1].content}")

print("\n--- 奇数のテスト ---")
inputs_odd = {"messages": [HumanMessage(content="77")]}
final_state_odd = graph.invoke(inputs_odd)
print(f"最終応答: {final_state_odd['messages'][-1].content}")
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class ConditionalGraphState(TypedDict):
    messages: Annotated[list, add_messages]
    number: int # 数値を保持する状態

# --- ノード定義 (Nodes) ---
def get_number_node(state: ConditionalGraphState):
    # ユーザーからのメッセージ（数値と期待）を抽出し、状態にセット
    last_message = state["messages"][-1].content
    print(f"get_number_node: 入力文字列 '{last_message}'")
    try:
        num = int(last_message)
        print(f"  -> 抽出された数値: {num}")
        return {"number": num}
    except ValueError:
        print(f"  -> 数値への変換失敗。デフォルト値0を使用します。")
        # エラーメッセージをmessagesに追加しても良い
        return {"number": 0, "messages": [AIMessage(content="無効な数値が入力されました。0として処理を続けます。")]}

def even_node(state: ConditionalGraphState):
    # 数値が偶数の場合の処理
    num = state["number"]
    result_message = f"入力された数値 {num} は偶数です。"
    print(f"even_node: {result_message}")
    return {"messages": [AIMessage(content=result_message)]}

def odd_node(state: ConditionalGraphState):
    # 数値が奇数の場合の処理
    num = state["number"]
    result_message = f"入力された数値 {num} は奇数です。"
    print(f"odd_node: {result_message}")
    return {"messages": [AIMessage(content=result_message)]}

# --- 条件付きエッジのルーター関数 ---
def number_router(state: ConditionalGraphState):
    # numberキーの値に基づいて次に遷移するノード名を返す
    num = state["number"]
    print(f"number_router: 数値 {num} でルーティング判断")
    if num % 2 == 0:
        print("  -> even_branch へ")
        return "even_branch" # add_conditional_edgesのマッピングキーと一致させる
    else:
        print("  -> odd_branch へ")
        return "odd_branch"

# --- グラフ構築 ---
workflow = StateGraph(ConditionalGraphState)

# ノードを追加
workflow.add_node("input_node", get_number_node)
workflow.add_node("even_processor", even_node)
workflow.add_node("odd_processor", odd_node)

# エントリポイントを設定
workflow.set_entry_point("input_node")

# 条件付きエッジを設定
# input_node の後に number_router を実行し、その戻り値に応じて分岐
workflow.add_conditional_edges(
    "input_node",
    number_router,
    {
        "even_branch": "even_processor", # routerが "even_branch" を返したら even_processor へ
        "odd_branch": "odd_processor"   # routerが "odd_branch" を返したら odd_processor へ
    }
)

# 各分岐の終点からENDへ
workflow.add_edge("even_processor", END)
workflow.add_edge("odd_processor", END)

# グラフをコンパイル
graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
print("\n--- 偶数のテスト (入力: 42) ---")
inputs_even = {"messages": [HumanMessage(content="42")]}
final_state_even = graph.invoke(inputs_even)
print(f"最終的な応答: {final_state_even['messages'][-1].content}")

print("\n--- 奇数のテスト (入力: 77) ---")
inputs_odd = {"messages": [HumanMessage(content="77")]}
final_state_odd = graph.invoke(inputs_odd)
print(f"最終的な応答: {final_state_odd['messages'][-1].content}")

print("\n--- 無効な入力のテスト (入力: abc) ---")
inputs_invalid = {"messages": [HumanMessage(content="abc")]}
final_state_invalid = graph.invoke(inputs_invalid)
print(f"最終的な応答 (エラー処理後): {final_state_invalid['messages'][-1].content}") # 最後のメッセージは偶数/奇数の結果
print(f"  中間メッセージ (エラー通知): {final_state_invalid['messages'][-2].content}") # エラー通知メッセージ
``````
</details>

## 問題002: 状態を使ったループ（シンプルなカウンター制御）

### 課題
グラフ内で特定の条件が満たされるまで処理を繰り返す「ループ」構造を、状態と条件付きエッジを使って実現する方法を学びましょう。ここでは、カウンターが指定した上限値に達するまで、カウンターを増やし続けるシンプルなループ処理を持つグラフを作成します。これは、より複雑な自己修正ループや反復処理の基礎となります。

*   **学習内容:** 状態(`State`)と条件付きエッジ(`add_conditional_edges`)を組み合わせて、グラフ内にループ構造（繰り返し処理）を実装する方法を学びます。具体的には、カウンターが上限に達するまで特定のノードを繰り返し実行し、条件を満たしたら別のノードに遷移して終了します。


In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class LoopState(TypedDict):
    messages: Annotated[list, add_messages]
    current_count: int
    max_count: int

# --- ノード定義 (Nodes) ---
def increment_node(state: LoopState):
    count = state.get("current_count", 0) + 1
    return {"current_count": count, "messages": [AIMessage(content=f"カウント: {count}")]}

def end_loop_node(state: LoopState):
    return {"messages": [AIMessage(content=f"ループ終了。最終カウント: {state['current_count']}")]}

# --- ルーター関数 ---
def loop_router(state: LoopState):
    if state["current_count"] < state["max_count"]:
        return "continue_loop"
    else:
        return "exit_loop"

# --- グラフ構築 ---
workflow = StateGraph(LoopState)
workflow.add_node("incrementer", increment_node)
workflow.add_node("loop_ender", end_loop_node)

workflow.set_entry_point("incrementer")
workflow.add_conditional_edges(
    "incrementer",
    loop_router,
    {
        "continue_loop": "incrementer",
        "exit_loop": "loop_ender"
    }
)
workflow.add_edge("loop_ender", END)

graph = workflow.compile()

# --- 実行 --- 
inputs = {"messages": [HumanMessage(content="ループ開始")], "current_count": 0, "max_count": 3}
for event in graph.stream(inputs, {"recursion_limit": 10}):
    print(event)
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class LoopState(TypedDict):
    messages: Annotated[list, add_messages]
    current_count: int # 現在のカウンター値
    max_count: int     # ループを終了するカウンターの上限値

# --- ノード定義 (Nodes) ---
def increment_and_log_node(state: LoopState):
    #カウンターを1増やし、メッセージをログに記録する
    # current_count が未定義(初回など)の場合、0として扱う
    count = state.get("current_count", 0) + 1
    log_message = f"カウンターが {count} に増加しました。"
    print(f"increment_and_log_node: {log_message}")
    return {"current_count": count, "messages": [AIMessage(content=log_message)]}

def final_log_node(state: LoopState):
    # ループ終了後に最終メッセージをログに記録する
    log_message = f"ループが終了しました。最終カウントは {state['current_count']} です。(上限: {state['max_count']})"
    print(f"final_log_node: {log_message}")
    return {"messages": [AIMessage(content=log_message)]}

# --- 条件付きエッジのルーター関数 ---
def should_continue_loop_router(state: LoopState):
    # current_countがmax_count未満ならループを継続、そうでなければ終了
    current = state.get("current_count", 0)
    max_c = state.get("max_count", 0)
    print(f"should_continue_loop_router: 現在カウント {current}, 上限カウント {max_c}")
    if current < max_c:
        print("  -> ループ継続 (increment_and_log_nodeへ)")
        return "continue_loop" # add_conditional_edgesのマッピングキーと一致
    else:
        print("  -> ループ終了 (final_log_nodeへ)")
        return "exit_loop"

# --- グラフ構築 ---
workflow = StateGraph(LoopState)

workflow.add_node("incrementer", increment_and_log_node)
workflow.add_node("final_logger", final_log_node)

# エントリポイントはカウンター増加ノード
# 初回実行時 current_count は入力で0に設定される想定
workflow.set_entry_point("incrementer")

# 条件付きエッジでループを制御
# incrementer ノードの後に should_continue_loop_router を実行
workflow.add_conditional_edges(
    "incrementer",
    should_continue_loop_router,
    {
        "continue_loop": "incrementer",  # "continue_loop"なら再度incrementerへ (ループバック)
        "exit_loop": "final_logger"    # "exit_loop"ならfinal_loggerへ
    }
)

# final_loggerノードからENDへ
workflow.add_edge("final_logger", END)

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
loop_max_count = 3
print(f"\n--- ループテスト (max_count = {loop_max_count}) ---")
inputs = {
    "messages": [HumanMessage(content=f"カウンターを{loop_max_count}回まで実行するループを開始")],
    "current_count": 0, # 初期カウントを0に設定
    "max_count": loop_max_count
}

print("\nストリーム出力:")
for event in graph.stream(inputs, {"recursion_limit": 10}): # ループがあるのでrecursion_limitに注意
    print(event)

print("\n最終状態の確認:")
final_state = graph.invoke(inputs, {"recursion_limit": 10})
print(f"  最終current_count: {final_state['current_count']}")
print(f"  最後のメッセージ: {final_state['messages'][-1].content}")
assert final_state['current_count'] == loop_max_count
assert f"最終カウントは {loop_max_count} です" in final_state['messages'][-1].content
print("アサーション成功！")
``````
</details>

## 問題003: LLM によるループ継続判断（自己反省ループ）

### 課題
問題002のカウンターループをより発展させ、LLMの判断に基づいてループを継続するかどうかを決定する「自己反省ループ」の基本的な形を実装します。ユーザーからの質問に対し、LLMが一度回答を生成し、その回答が十分かどうかを別のLLM（または同じLLMに異なるプロンプトで）が判断し、不十分なら改善を試みる（再度回答生成に戻る）という流れを作ります。ここでは、最大試行回数も設けます。

*   **学習内容:** LLMの出力を評価し、その評価結果に基づいて処理をループさせる方法を学びます。これは、ReAct（Reasoning and Acting）エージェントや、より複雑な反復的改善プロセスの基礎となります。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, List, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

# --- 状態定義 (State) ---
class SelfReflectionState(TypedDict):
    messages: Annotated[list, add_messages]
    original_question: str
    current_answer: Optional[str]
    is_sufficient: Optional[bool]
    attempt_count: int
    max_attempts: int

# --- ノード定義 (Nodes) ---
def generate_answer_node(state: SelfReflectionState):
    question = state["original_question"]
    attempt = state["attempt_count"] + 1
    print(f"generate_answer_node (Attempt {attempt}): Answering '{question}'")
    # 実際にはLLMで回答生成
    # response = llm.invoke([HumanMessage(content=question)])
    # generated_ans = response.content
    generated_ans = f"これが'{question}'に対する答えです。(試行{attempt}回目)"
    if attempt > 1: # 2回目以降は少し変えるデモ
        generated_ans += " 前回の回答を改善しました。"
    return {"current_answer": generated_ans, "attempt_count": attempt, "messages": [AIMessage(content=generated_ans)]}

def evaluate_answer_node(state: SelfReflectionState):
    answer = state["current_answer"]
    print(f"evaluate_answer_node: Evaluating '{answer}'")
    # 実際にはLLMで評価 (ここではダミー評価)
    # sufficient = llm.invoke("この回答は十分ですか？Yes/No: " + answer).content.startswith("Yes")
    sufficient = "改善しました" in answer or state["attempt_count"] >= state["max_attempts"] # ダミー: 改善したか最大試行回数なら十分
    return {"is_sufficient": sufficient, "messages": [AIMessage(content=f"評価結果: {'十分' if sufficient else '不十分'}")]}

# --- ルーター関数 ---
def reflection_router(state: SelfReflectionState):
    if state.get("is_sufficient") or state["attempt_count"] >= state["max_attempts"]:
        return "end_loop"
    else:
        return "regenerate_answer"

# --- グラフ構築 ---
workflow = StateGraph(SelfReflectionState)
workflow.add_node("answer_generator", generate_answer_node)
workflow.add_node("answer_evaluator", evaluate_answer_node)

workflow.set_entry_point("answer_generator")
workflow.add_edge("answer_generator", "answer_evaluator")
workflow.add_conditional_edges(
    "answer_evaluator",
    reflection_router,
    {
        "regenerate_answer": "answer_generator",
        "end_loop": END
    }
)
graph = workflow.compile()

# --- 実行 --- 
inputs = {
    "messages": [HumanMessage(content="LangGraphとは何ですか？")],
    "original_question": "LangGraphとは何ですか？",
    "attempt_count": 0,
    "max_attempts": 2
}
for event in graph.stream(inputs, {"recursion_limit": 10}):
    print(event)
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, List, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import Image, display

# --- 状態定義 (State) ---
class SelfReflectionState(TypedDict):
    messages: Annotated[list, add_messages] # 会話履歴
    original_question: str                 # ユーザーの最初の質問
    current_answer: Optional[str]          # 現在生成されている回答
    is_sufficient: Optional[bool]          # 回答が十分かどうかの評価結果
    attempt_count: int                     # 回答生成の試行回数
    max_attempts: int                      # 最大試行回数

# --- ノード定義 (Nodes) ---
def initialize_state_node(state: SelfReflectionState):
    # 最初のユーザーメッセージを original_question にセット
    # attempt_count を初期化
    user_question = state["messages"][-1].content
    print(f"initialize_state_node: 初期質問 '{user_question}' を設定。試行回数0。")
    return {
        "original_question": user_question,
        "attempt_count": 0,
        "current_answer": None,
        "is_sufficient": None
    }

def generate_answer_node(state: SelfReflectionState):
    question = state["original_question"]
    attempt = state.get("attempt_count", 0) + 1 # 試行回数をインクリメント
    print(f"generate_answer_node (試行 {attempt}/{state['max_attempts']}): 質問 '{question}' に回答生成中...")

    # LLMプロンプトの準備
    prompt_messages = [SystemMessage(content="あなたは質問応答AIです。簡潔かつ正確に答えてください。")]
    if attempt > 1 and state.get("current_answer"):
        prompt_messages.append(AIMessage(content=f"前回のあなたの回答: {state['current_answer']}"))
        prompt_messages.append(HumanMessage(content=f"その回答は不十分でした。より良い回答を生成してください。質問: {question}"))
    else:
        prompt_messages.append(HumanMessage(content=question))
    
    response = llm.invoke(prompt_messages) # llmはノートブック冒頭で初期化済み
    generated_ans = response.content.strip()
    print(f"  -> 生成された回答: '{generated_ans}'")
    
    return {
        "current_answer": generated_ans,
        "attempt_count": attempt,
        "messages": [AIMessage(content=generated_ans)] # 生成された回答を履歴に追加
    }

def evaluate_answer_node(state: SelfReflectionState):
    answer = state["current_answer"]
    question = state["original_question"]
    print(f"evaluate_answer_node: 回答 '{answer}' を評価中...")

    # 評価用LLMプロンプト (ここではダミー評価ロジックで代替)
    # 実際には、「この回答は質問に十分答えていますか？ Yes/No/Maybe と理由を述べてください」のようなプロンプトでLLMに評価させる
    # sufficient_eval = llm.invoke(eval_prompt).content

    # ダミー評価ロジック: 試行回数が少ないうちは「不十分」とし、改善を促す
    sufficient = False
    if state["attempt_count"] >= state["max_attempts"]:
        sufficient = True # 最大試行回数に達したら強制的に「十分」とする
        eval_message = "最大試行回数に達したため、これで十分とします。"
    elif "langgraph" in answer.lower() and "graph" in answer.lower(): # 簡単なキーワードチェック
        sufficient = True
        eval_message = "回答は質問に関連しており、十分と判断します。"
    else:
        eval_message = "回答はまだ不十分です。改善の余地があります。"
        
    print(f"  -> 評価結果: {'十分' if sufficient else '不十分'}. {eval_message}")
    return {"is_sufficient": sufficient, "messages": [AIMessage(content=f"評価コメント: {eval_message}")]}

def final_result_node(state: SelfReflectionState):
    final_answer = state["current_answer"]
    print(f"final_result_node: 最終的な回答は '{final_answer}' です。")
    # 最終回答をmessagesに追加する（既に追加されているかもしれないが、念のため）
    # 実際には、generate_answer_nodeで既に追加されているので、ここでは不要かもしれない
    # return {"messages": [AIMessage(content=f"最終決定された回答: {final_answer}")]}
    return {}

# --- ルーター関数 ---
def reflection_router(state: SelfReflectionState):
    print(f"reflection_router: 試行回数 {state['attempt_count']}/{state['max_attempts']}, 回答は十分か？ {state.get('is_sufficient')}")
    if state.get("is_sufficient") or state["attempt_count"] >= state["max_attempts"]:
        print("  -> ループ終了 (final_result_nodeへ)")
        return "end_reflection_loop"
    else:
        print("  -> 回答再生成 (generate_answer_nodeへ)")
        return "regenerate_answer"

# --- グラフ構築 ---
workflow = StateGraph(SelfReflectionState)

workflow.add_node("initializer", initialize_state_node)
workflow.add_node("answer_generator", generate_answer_node)
workflow.add_node("answer_evaluator", evaluate_answer_node)
workflow.add_node("final_result", final_result_node)

workflow.set_entry_point("initializer")
workflow.add_edge("initializer", "answer_generator") # 初期化後、最初の回答生成へ
workflow.add_edge("answer_generator", "answer_evaluator") # 回答生成後、評価へ

workflow.add_conditional_edges(
    "answer_evaluator", # 評価ノードの後で分岐
    reflection_router,    # ルーター関数
    {
        "regenerate_answer": "answer_generator", # 不十分なら再度回答生成へ (ループ)
        "end_reflection_loop": "final_result"      # 十分なら最終結果ノードへ
    }
)
workflow.add_edge("final_result", END) # 最終結果の後、終了

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
user_query = "LangGraphとは何ですか？簡単に教えてください。"
max_reflection_attempts = 2 # 最大試行回数 (最初の1回 + 改善1回)

print(f"\n--- 自己反省ループテスト (質問: '{user_query}', 最大試行: {max_reflection_attempts}) ---")
inputs = {
    "messages": [HumanMessage(content=user_query)],
    "max_attempts": max_reflection_attempts
    # original_question, attempt_count などは initializer ノードで設定される
}

print("\nストリーム出力:")
for event in graph.stream(inputs, {"recursion_limit": 15}): # ループ回数に応じてrecursion_limit調整
    print(event)

print("\n最終状態の確認 (invoke):")
final_state = graph.invoke(inputs, {"recursion_limit": 15})
print(f"  元の質問: {final_state['original_question']}")
print(f"  最終的な回答: {final_state['current_answer']}")
print(f"  試行回数: {final_state['attempt_count']}")
print(f"  回答は十分か: {final_state['is_sufficient']}")
print(f"  最後のメッセージ(AI): {final_state['messages'][-1].content if final_state['messages'] and isinstance(final_state['messages'][-1], AIMessage) else 'N/A'}")
``````
</details>

## 問題004: 並列処理（ファンアウト・ファンイン）による複数タスクの同時実行

### 課題
あるタスク（例えば文章の入力）の後、後続する複数の異なるタスクを同時に実行したい場合があります。LangGraphでは、**1つのノードから、並列で実行したい複数の後続ノードへ個別にエッジを接続する**ことで、処理を分岐させ、複数のタスクを並列で実行（ファンアウト）できます。

さらに、並列実行したすべてのタスクが完了するのを待ってから結果を1つに統合（ファンイン）するには、`add_edge`メソッドの**第1引数**にノード名のリストを渡します。

この問題では、与えられた文章に対して『要約作成』と『キーワード抽出』という2つの処理を並列で行い、最後に両方の結果が揃ってから統合する、という一連のグラフを構築します。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, List, Optional, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages # メッセージログ用に
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class FanOutFanInState(TypedDict):
    messages: Annotated[list, add_messages]
    original_text: str
    summary: Optional[str]
    keywords: Optional[List[str]]
    final_report: Optional[str]

# --- ノード定義 (Nodes) ---
def get_text_node(state: FanOutFanInState):
    text = state["messages"][-1].content
    return {"original_text": text}

def summarize_node(state: FanOutFanInState):
    text = state["original_text"]
    # result = llm.invoke(f"要約してください: {text}").content
    result = f"「{text[:10]}...」の要約です。"
    return {"summary": result, "messages": [AIMessage(content=f"要約完了: {result}")]}

def extract_keywords_node(state: FanOutFanInState):
    text = state["original_text"]
    # kws = llm.invoke(f"キーワードを抽出: {text}").content.split(',')
    kws = [f"キーワード{i+1}" for i in range(min(3, len(text.split())))]
    return {"keywords": kws, "messages": [AIMessage(content=f"キーワード抽出完了: {kws}")]}

def aggregate_node(state: FanOutFanInState):
    report = f"要約: {state['summary']}\nキーワード: {state['keywords']}"
    return {"final_report": report, "messages": [AIMessage(content=f"最終レポート生成完了。\n{report}")]}

# --- グラフ構築 ---
workflow = StateGraph(FanOutFanInState)
workflow.add_node("input_text_getter", get_text_node)
workflow.add_node("summarizer", summarize_node)
workflow.add_node("keyword_extractor", extract_keywords_node)
workflow.add_node("aggregator", aggregate_node)

workflow.set_entry_point("input_text_getter")
workflow.add_edge("input_text_getter", "summarizer")
workflow.add_edge("input_text_getter", "keyword_extractor") # ファンアウト
workflow.add_edge(["summarizer", "keyword_extractor"], "aggregator") # ファンイン
workflow.add_edge("aggregator", END)

graph = workflow.compile()

# --- 実行 --- 
inputs = {"messages": [HumanMessage(content="LangGraphはグラフベースの処理フローを簡単に作れます。")]}
for event in graph.stream(inputs):
    print(event)
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, List, Optional, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages # メッセージログ用に
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class FanOutFanInState(TypedDict):
    messages: Annotated[list, add_messages] # 処理のログやデバッグ情報用
    original_text: str                   # 入力される元のテキスト
    summary: Optional[str]                 # 要約結果を格納
    keywords: Optional[List[str]]          # キーワードリストを格納
    final_report: Optional[str]            # 最終的な統合レポート

# --- ノード定義 (Nodes) ---
def get_text_node(state: FanOutFanInState):
    # messagesから最新のユーザー入力をoriginal_textに設定
    # このノードがエントリポイントで、最初の入力はmessages経由で渡される想定
    text_input = state["messages"][-1].content
    print(f"get_text_node: 入力テキスト '{text_input}' を取得しました。")
    return {"original_text": text_input}

def summarize_node(state: FanOutFanInState):
    text_to_summarize = state["original_text"]
    print(f"summarize_node: テキスト '{text_to_summarize[:20]}...' の要約を開始します。")
    # ここではダミーの要約処理（実際にはLLMなどを使用）
    # summary_result = llm.invoke(f"この文章を要約してください: {text_to_summarize}").content
    summary_result = f"これは「{text_to_summarize[:15]}...」に関する要約です。"
    print(f"  -> 要約結果: '{summary_result}'")
    return {"summary": summary_result, "messages": [AIMessage(content=f"要約処理完了: {summary_result}")]}

def extract_keywords_node(state: FanOutFanInState):
    text_to_extract = state["original_text"]
    print(f"extract_keywords_node: テキスト '{text_to_extract[:20]}...' のキーワード抽出を開始します。")
    # ここではダミーのキーワード抽出処理
    # keywords_result = llm.invoke(f"この文章からキーワードを3つ抽出してください: {text_to_extract}").content.split(',')
    keywords_result = [f"キーワード{i+1}" for i in range(min(3, len(text_to_extract.split())))] # ダミー
    print(f"  -> 抽出キーワード: {keywords_result}")
    return {"keywords": keywords_result, "messages": [AIMessage(content=f"キーワード抽出完了: {keywords_result}")]}

def aggregate_results_node(state: FanOutFanInState):
    summary = state.get("summary", "(要約なし)")
    keywords = state.get("keywords", [])
    print(f"\naggregate_results_node: 要約とキーワードを統合します...")
    print(f"  取得した要約: {summary}")
    print(f"  取得したキーワード: {keywords}")
    
    report = f"## 分析レポート\n\n### 要約\n{summary}\n\n### 主要キーワード\n- {'\n- '.join(keywords)}"
    print(f"  -> 生成されたレポート:\n{report}")
    return {"final_report": report, "messages": [AIMessage(content=f"最終レポート生成完了。\n{report}")]}

# --- グラフ構築 ---
workflow = StateGraph(FanOutFanInState)

workflow.add_node("input_text_getter", get_text_node)
workflow.add_node("summarizer", summarize_node)
workflow.add_node("keyword_extractor", extract_keywords_node)
workflow.add_node("aggregator", aggregate_results_node)

workflow.set_entry_point("input_text_getter")

# ファンアウト: input_text_getter から summarizer と keyword_extractor へ並列実行
workflow.add_edge("input_text_getter", "summarizer")
workflow.add_edge("input_text_getter", "keyword_extractor")

# ファンイン: summarizer と keyword_extractor の両方が完了したら aggregator へ
workflow.add_edge(["summarizer", "keyword_extractor"], "aggregator")

workflow.add_edge("aggregator", END)

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
input_sentence = "LangGraphは、LLMアプリケーション構築のためのライブラリです。状態を持つグラフとして複雑な処理フローを定義できます。並列処理やループ、条件分岐など、高度な制御が可能です。"
print(f"\n--- 並列処理（ファンアウト・ファンイン）テスト (入力: '{input_sentence}') ---")
inputs = {"messages": [HumanMessage(content=input_sentence)]}

print("\nストリーム出力:")
for event in graph.stream(inputs, {"recursion_limit": 10}):
    print(event)

print("\n最終状態の確認 (invoke):")
final_state = graph.invoke(inputs, {"recursion_limit": 10})
print(f"  元のテキスト: {final_state['original_text']}")
print(f"  要約: {final_state['summary']}")
print(f"  キーワード: {final_state['keywords']}")
print(f"  最終レポート:\n{final_state['final_report']}")
assert final_state['summary'] is not None
assert final_state['keywords'] is not None
assert final_state['final_report'] is not None
print("アサーション成功！")
``````
</details>

## 問題005: Human-in-the-Loop (人間による介在と承認)

### 課題
自動処理の途中で人間の判断や承認を挟む「Human-in-the-Loop」は、AIシステムの信頼性や安全性を高める上で重要です。LangGraphでは、特定のノードで処理を一時停止し、ユーザーからの入力を待ってから再開する仕組みを構築できます。ここでは、LLMが生成した文章をユーザーが確認・承認し、承認されれば次の処理へ、されなければ修正を促す（または終了する）というフローを作成します。

*   **学習内容:** `graph.update_state()` や `graph.get_state()` を利用して、外部（この場合はユーザーの`input()`）からグラフの状態を更新し、処理を再開する方法を学びます。中断と再開の制御には、`Interrupt` 例外と `compile(checkpointer=...)` が関連しますが、この問題ではより基本的な `input()` による同期的な待機と状態更新で概念を理解します。（非同期な中断・再開は第5章で扱います）

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class HumanApprovalState(TypedDict):
    messages: Annotated[list, add_messages]
    generated_text: Optional[str]
    user_approval: Optional[bool] # True: 承認, False: 拒否, None: 未確認
    feedback: Optional[str] # ユーザーからのフィードバック

# --- ノード定義 (Nodes) ---
def generate_text_node(state: HumanApprovalState):
    # ユーザーメッセージからトピックを取得して文章生成（ダミー）
    topic = state["messages"][-1].content if state["messages"] and isinstance(state["messages"][-1], HumanMessage) else "何か"
    text = f"「{topic}」に関する重要な提案書です。ご確認ください。"
    print(f"generate_text_node: 生成されたテキスト: '{text}'")
    return {"generated_text": text, "user_approval": None, "feedback": None, "messages":[AIMessage(content=text)]}

def human_approval_node(state: HumanApprovalState):
    generated_text = state["generated_text"]
    print("\n--- 人間による確認 --- ")
    print(f"生成されたテキスト: {generated_text}")
    
    approval_input = input("この内容で承認しますか？ (yes/no): ").lower()
    user_approved = approval_input == "yes"
    
    user_feedback = None
    if not user_approved:
        user_feedback = input("修正点やフィードバックがあれば入力してください: ")
        
    return {"user_approval": user_approved, "feedback": user_feedback, "messages": [HumanMessage(content=f"承認状態: {'承認' if user_approved else '拒否'}. フィードバック: {user_feedback or 'なし'}")]}

def process_approved_node(state: HumanApprovalState):
    print("process_approved_node: 承認されたため、次の処理を実行します。")
    return {"messages":[AIMessage(content="承認されました。処理を継続します。")]}

def process_rejected_node(state: HumanApprovalState):
    feedback = state.get("feedback")
    print(f"process_rejected_node: 拒否されました。フィードバック: '{feedback}'")
    # ここで修正プロセスに入るか、終了するかなどを実装できる
    return {"messages":[AIMessage(content=f"拒否されました。フィードバックに基づいて修正が必要です: {feedback}")]}

# --- ルーター関数 ---
def approval_router(state: HumanApprovalState):
    if state.get("user_approval") is True:
        return "approved"
    else: # False または None (ありえないが念のため)
        return "rejected"

# --- グラフ構築 ---
workflow = StateGraph(HumanApprovalState)
workflow.add_node("text_generator", generate_text_node)
workflow.add_node("human_validator", human_approval_node)
workflow.add_node("approved_handler", process_approved_node)
workflow.add_node("rejected_handler", process_rejected_node)

workflow.set_entry_point("text_generator")
workflow.add_edge("text_generator", "human_validator")
workflow.add_conditional_edges(
    "human_validator",
    approval_router,
    {
        "approved": "approved_handler",
        "rejected": "rejected_handler"
    }
)
workflow.add_edge("approved_handler", END)
workflow.add_edge("rejected_handler", END) # この例では拒否後も終了

graph = workflow.compile()

# --- 実行 --- 
print("--- Human-in-the-Loop テスト (承認ケース) ---")
# ユーザー入力 'yes' を想定
inputs_approve = {"messages": [HumanMessage(content="新製品のローンチ計画")]}
final_state_approve = graph.invoke(inputs_approve)
print(f"最終メッセージ (承認): {final_state_approve['messages'][-1].content}")

print("\n--- Human-in-the-Loop テスト (拒否ケース) ---")
# ユーザー入力 'no', フィードバック '予算が不足しています' を想定
inputs_reject = {"messages": [HumanMessage(content="マーケティング戦略")]}
final_state_reject = graph.invoke(inputs_reject)
print(f"最終メッセージ (拒否): {final_state_reject['messages'][-1].content}")
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, Optional
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class HumanApprovalState(TypedDict):
    messages: Annotated[list, add_messages] # 会話履歴や処理ログ
    generated_text: Optional[str]          # AIが生成したテキスト
    user_approval: Optional[bool]          # True: 承認, False: 拒否, None: 未確認
    feedback: Optional[str]                # ユーザーからのフィードバック（拒否の場合など）
    task_description: Optional[str]        # 元のタスク記述

# --- ノード定義 (Nodes) ---
def generate_text_node(state: HumanApprovalState):
    # ユーザーからの最新のメッセージをタスク記述として取得
    task_desc = state["messages"][-1].content if state["messages"] and isinstance(state["messages"][-1], HumanMessage) else "デフォルトタスク"
    print(f"generate_text_node: タスク記述 '{task_desc}' に基づいてテキストを生成します。")
    
    # ダミーのテキスト生成（実際にはLLM呼び出し）
    # generated_content = llm.invoke(f"以下のタスク記述に基づいて提案書を作成してください: {task_desc}").content
    generated_content = f"これは「{task_desc}」に関する提案書（案）です。ご確認をお願いいたします。"
    print(f"  -> 生成されたテキスト: '{generated_content}'")
    
    return {
        "generated_text": generated_content,
        "task_description": task_desc,
        "user_approval": None, # 承認状態をリセット
        "feedback": None,      # フィードバックをリセット
        "messages": [AIMessage(content=f"生成された提案書案: {generated_content}")]
    }

def human_approval_node(state: HumanApprovalState):
    generated_text = state.get("generated_text", "(テキストが生成されていません)")
    print("\n--- 人間による確認ステップ --- ")
    print(f"タスク: {state.get('task_description')}")
    print(f"生成されたテキスト案:\n{'-'*30}\n{generated_text}\n{'-'*30}")
    
    approval_input = ""
    while approval_input not in ["yes", "no"]:
        approval_input = input("この内容で承認しますか？ (yes/no): ").strip().lower()
    
    user_approved = (approval_input == "yes")
    user_feedback_text = None
    
    if user_approved:
        print("  -> 承認されました。")
        approval_message = "ユーザーによって承認されました。"
    else:
        print("  -> 拒否されました。")
        user_feedback_text = input("修正のためのフィードバックを入力してください (任意): ").strip()
        if not user_feedback_text:
            user_feedback_text = "(フィードバックなし)"
        print(f"  -> 受け取ったフィードバック: '{user_feedback_text}'")
        approval_message = f"ユーザーによって拒否されました。フィードバック: {user_feedback_text}"
        
    return {
        "user_approval": user_approved,
        "feedback": user_feedback_text,
        "messages": [HumanMessage(content=approval_message)] # 人間のアクションを履歴に追加
    }

def process_approved_text_node(state: HumanApprovalState):
    approved_text = state["generated_text"]
    print(f"process_approved_text_node: 承認されたテキスト '{approved_text[:30]}...' を使って最終処理を実行します。")
    # ここで承認後の処理（例: 保存、送信など）を行う
    final_message = f"テキスト「{approved_text[:20]}...」は承認され、処理が完了しました。"
    return {"messages":[AIMessage(content=final_message)]}

def handle_rejected_text_node(state: HumanApprovalState):
    feedback = state.get("feedback", "(フィードバックなし)")
    rejected_text = state.get("generated_text", "(不明なテキスト)")
    print(f"handle_rejected_text_node: テキスト '{rejected_text[:30]}...' は拒否されました。フィードバック: '{feedback}'")
    # この例では、拒否されたら修正プロセスには戻らずに終了する
    # より高度なフローでは、ここから再度 generate_text_node に戻るループを組むことも可能
    rejection_message = f"テキスト「{rejected_text[:20]}...」は拒否されました。フィードバック: 「{feedback}」。処理を終了します。"
    return {"messages":[AIMessage(content=rejection_message)]}

# --- ルーター関数 ---
def route_after_approval(state: HumanApprovalState):
    print(f"route_after_approval: ユーザー承認状態 -> {state.get('user_approval')}")
    if state.get("user_approval") is True:
        print("  -> approved_handler へ")
        return "was_approved"
    else: # False または None (エラーケースだが分岐は明確に)
        print("  -> rejected_handler へ")
        return "was_rejected"

# --- グラフ構築 ---
workflow = StateGraph(HumanApprovalState)

workflow.add_node("text_generator", generate_text_node)
workflow.add_node("human_validator", human_approval_node)
workflow.add_node("approved_handler", process_approved_text_node)
workflow.add_node("rejected_handler", handle_rejected_text_node)

workflow.set_entry_point("text_generator")
workflow.add_edge("text_generator", "human_validator")

workflow.add_conditional_edges(
    "human_validator",
    route_after_approval,
    {
        "was_approved": "approved_handler",
        "was_rejected": "rejected_handler"
    }
)
workflow.add_edge("approved_handler", END)
workflow.add_edge("rejected_handler", END) # この例では拒否された場合も終了

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
print("\n--- Human-in-the-Loop テスト (ユーザーが 'yes' と入力する想定) ---")
inputs_for_approval = {"messages": [HumanMessage(content="新製品のキャッチコピー案作成")]}
final_state_approved = graph.invoke(inputs_for_approval)
print(f"最終メッセージ (承認時): {final_state_approved['messages'][-1].content}")

print("\n--- Human-in-the-Loop テスト (ユーザーが 'no' と入力し、フィードバックする想定) ---")
inputs_for_rejection = {"messages": [HumanMessage(content="顧客向け謝罪文案作成")]}
final_state_rejected = graph.invoke(inputs_for_rejection)
print(f"最終メッセージ (拒否時): {final_state_rejected['messages'][-1].content}")

assert "承認されました" in final_state_approved['messages'][-1].content 
assert "拒否されました" in final_state_rejected['messages'][-1].content
print("\nアサーション成功！")
``````
</details>

## 問題006: エラーハンドリングとリトライ処理

### 課題
グラフ内のノード処理（特に外部API呼び出しなど）では、一時的なエラーが発生することがあります。このような場合、即座に処理を失敗させるのではなく、数回リトライ（再試行）するメカニズムを組み込むことが有効です。この問題では、エラーが発生した場合に最大N回まで処理をリトライし、それでも成功しない場合にエラーとして処理を終了するグラフを作成します。

*   **学習内容:** ノード内でエラーを捕捉し、状態にリトライ回数を記録します。条件付きエッジを使って、リトライ回数が上限未満であれば再度同じノードを実行（リトライ）し、上限に達したら別のエラー処理ノードへ分岐する方法を学びます。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, Optional
import random # リトライ成功をシミュレートするため
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage

# --- 状態定義 (State) ---
class RetryState(TypedDict):
    messages: Annotated[list, add_messages]
    task_input: str
    result: Optional[str]
    error_message: Optional[str]
    retry_count: int
    max_retries: int

# --- ノード定義 (Nodes) ---
def unreliable_task_node(state: RetryState):
    current_retry = state.get("retry_count", 0)
    print(f"unreliable_task_node (試行 {current_retry + 1}/{state['max_retries']}): タスク '{state['task_input']}' を実行中...")
    # 確率的に成功/失敗をシミュレート (例: 50%の確率で失敗)
    if random.random() < 0.5 and current_retry < state['max_retries'] -1 : # 最初の方は失敗しやすくする
        err_msg = "一時的なネットワークエラーが発生しました。"
        print(f"  -> 失敗: {err_msg}")
        return {"error_message": err_msg, "retry_count": current_retry + 1, "messages":[AIMessage(content=f"試行失敗: {err_msg}")]}
    else:
        res = f"タスク '{state['task_input']}' の処理成功。(試行 {current_retry + 1}回目)"
        print(f"  -> 成功: {res}")
        return {"result": res, "error_message": None, "retry_count": current_retry + 1, "messages":[AIMessage(content=res)]}

def handle_failure_node(state: RetryState):
    final_err_msg = f"タスク '{state['task_input']}' は最大リトライ回数 ({state['max_retries']}) を超えても成功しませんでした。最終エラー: {state.get('error_message')}"
    print(f"handle_failure_node: {final_err_msg}")
    return {"messages":[AIMessage(content=final_err_msg)]}

# --- ルーター関数 ---
def retry_router(state: RetryState):
    if state.get("result") is not None: # 成功時
        print("retry_router: タスク成功。終了します。")
        return "task_succeeded"
    elif state.get("retry_count", 0) < state.get("max_retries", 0):
        print(f"retry_router: リトライ可能 (現在 {state['retry_count']}/{state['max_retries']})。再試行します。")
        return "retry_task"
    else: # リトライ上限超え
        print("retry_router: リトライ上限超過。エラー処理へ。")
        return "max_retries_exceeded"

# --- グラフ構築 ---
workflow = StateGraph(RetryState)
workflow.add_node("flaky_task_runner", unreliable_task_node)
workflow.add_node("failure_handler", handle_failure_node)

workflow.set_entry_point("flaky_task_runner")
workflow.add_conditional_edges(
    "flaky_task_runner",
    retry_router,
    {
        "task_succeeded": END,
        "retry_task": "flaky_task_runner", # 自分自身に戻ってリトライ
        "max_retries_exceeded": "failure_handler"
    }
)
workflow.add_edge("failure_handler", END)
graph = workflow.compile()

# --- 実行 --- 
print("--- エラーハンドリングとリトライテスト (3回リトライ) ---")
inputs = {
    "messages": [HumanMessage(content="不安定なAPIを呼び出すタスク")],
    "task_input": "不安定なAPI呼び出し",
    "retry_count": 0,
    "max_retries": 3
}
for event in graph.stream(inputs, {"recursion_limit": 15}):
    print(event)

final_state = graph.invoke(inputs, {"recursion_limit": 15})
print(f"\n最終結果メッセージ: {final_state['messages'][-1].content}")
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, Optional
import random # リトライ成功をシミュレートするため
import time   # リトライ間の待機時間のため
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class RetryState(TypedDict):
    messages: Annotated[list, add_messages]
    task_input: str                  # 処理対象の入力
    result: Optional[str]            # 処理成功時の結果
    error_message: Optional[str]     # 直近のエラーメッセージ
    attempt_count: int               # 現在の試行回数 (0から開始)
    max_attempts: int                # 最大試行回数

# --- ノード定義 (Nodes) ---
def initialize_retry_state_node(state: RetryState):
    # messagesから最初のユーザー入力をtask_inputに設定
    # attempt_countを初期化
    user_input_content = state["messages"][-1].content
    print(f"initialize_retry_state_node: タスク入力 '{user_input_content}' を設定。試行回数0。")
    return {
        "task_input": user_input_content,
        "attempt_count": 0,
        "result": None,
        "error_message": None
    }

def unreliable_task_node(state: RetryState):
    current_attempt = state.get("attempt_count", 0) + 1 # この試行が何回目か
    max_attempts_val = state.get("max_attempts", 1)
    task_data = state["task_input"]
    
    print(f"unreliable_task_node (試行 {current_attempt}/{max_attempts_val}): タスク '{task_data}' を実行中...")
    
    # 確率的に成功/失敗をシミュレート
    # 例: 試行回数が少ないほど失敗しやすく、回数が増えると成功しやすくなる（またはランダム）
    # ここでは、最後の試行以外は失敗するかもしれない、というシナリオを簡単に作る
    if current_attempt < max_attempts_val and random.random() < 0.7: # 70%の確率で失敗 (最後の試行でなければ)
        err_msg = f"一時的なエラーが発生しました (例: タイムアウト)。試行 {current_attempt}"
        print(f"  -> 失敗: {err_msg}")
        # time.sleep(1) # 実際のリトライでは待機時間を挟むことが多い
        return {"error_message": err_msg, "attempt_count": current_attempt, "messages":[AIMessage(content=f"タスク試行 {current_attempt} 失敗: {err_msg}")]}
    else:
        # 成功ケース
        res = f"タスク '{task_data}' の処理成功。(試行 {current_attempt}回目)"
        print(f"  -> 成功: {res}")
        return {"result": res, "error_message": None, "attempt_count": current_attempt, "messages":[AIMessage(content=res)]}

def handle_permanent_failure_node(state: RetryState):
    final_err_msg = (
        f"タスク '{state['task_input']}' は最大試行回数 ({state['max_attempts']}) を超えても成功しませんでした。\n"
        f"最終エラー: {state.get('error_message', '(不明なエラー)')}"
    )
    print(f"handle_permanent_failure_node: {final_err_msg}")
    # ここで、エラーをログに記録したり、ユーザーに通知したりする処理を追加できる
    return {"messages":[AIMessage(content=final_err_msg)]}

# --- ルーター関数 ---
def retry_or_fail_router(state: RetryState):
    current_attempt = state.get("attempt_count", 0)
    max_attempts_val = state.get("max_attempts", 1)
    
    print(f"retry_or_fail_router: 試行 {current_attempt}/{max_attempts_val}. 結果存在: {state.get('result') is not None}")
    
    if state.get("result") is not None: # resultキーに何か値があれば成功とみなす
        print("  -> タスク成功。ENDへ")
        return "task_succeeded_end"
    elif current_attempt < max_attempts_val:
        print(f"  -> リトライ可能。unreliable_task_nodeへ戻る")
        return "needs_retry"
    else: # リトライ上限超過
        print("  -> リトライ上限超過。handle_permanent_failure_nodeへ")
        return "max_retries_exceeded_fail"

# --- グラフ構築 ---
workflow = StateGraph(RetryState)

workflow.add_node("initializer", initialize_retry_state_node)
workflow.add_node("flaky_task_runner", unreliable_task_node)
workflow.add_node("failure_handler", handle_permanent_failure_node)

workflow.set_entry_point("initializer")
workflow.add_edge("initializer", "flaky_task_runner") # 初期化後、タスク実行へ

workflow.add_conditional_edges(
    "flaky_task_runner", # このノードの後に分岐
    retry_or_fail_router,  # ルーター関数
    {
        "task_succeeded_end": END,              # 成功したら終了
        "needs_retry": "flaky_task_runner",   # リトライが必要なら再度同じタスクへ (ループ)
        "max_retries_exceeded_fail": "failure_handler" # 上限超えならエラーハンドラへ
    }
)
workflow.add_edge("failure_handler", END) # エラーハンドラの後も終了

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行と結果確認 ---
max_retries_setting = 3
print(f"\n--- エラーハンドリングとリトライテスト (最大リトライ: {max_retries_setting}回) ---")
inputs = {
    "messages": [HumanMessage(content="外部APIへのデータ送信タスク")],
    "max_attempts": max_retries_setting
    # task_input, attempt_count などは initializer ノードで設定
}

print("\nストリーム出力:")
for event in graph.stream(inputs, {"recursion_limit": max_retries_setting * 2 + 5 }): # リトライ回数に応じて調整
    print(event)

print("\n最終状態の確認 (invoke):")
final_state = graph.invoke(inputs, {"recursion_limit": max_retries_setting * 2 + 5 })
print(f"  タスク入力: {final_state['task_input']}")
print(f"  最終試行回数: {final_state['attempt_count']}")
if final_state.get('result'):
    print(f"  結果: {final_state['result']}")
else:
    print(f"  エラーメッセージ: {final_state.get('error_message')}")
print(f"  最後のAIメッセージ: {final_state['messages'][-1].content if final_state['messages'] else 'N/A'}")
``````
</details>

## 問題007: 第2章のまとめ - 反復的な改善と最終承認フローの構築

### 課題
第2章で学んだ制御フローの要素（条件分岐、ループ、人間による介在、エラーハンドリングとリトライ）を組み合わせて、より複雑なワークフローを構築します。具体的には、以下のステップを含む「反復的な改善と最終承認フロー」を作成します。
1.  **初期提案生成:** ユーザーからの指示に基づき、LLMが初期提案（例: 文章案、計画案など）を生成します。
2.  **自己評価と改善ループ:** 生成された提案をLLM自身が（または別の評価用LLMが）評価します。評価基準を満たさなければ、最大N回まで改善を試みます（ループ）。
3.  **人間による最終承認:** 自己改善ループを経た提案を人間が確認し、承認または差し戻し（フィードバック付き）を決定します。
4.  **処理完了または修正へ:** 承認されれば処理完了。差し戻された場合は、フィードバックを元に再度提案生成からやり直すか、あるいはエラーとして終了します（この問題では簡略化のため、差し戻し後はエラー終了とします）。
途中で予期せぬエラーが発生した場合は、リトライ処理も挟みます（簡易的なもの）。

*   **学習内容:** これまでに学んだ制御フローの知識を総動員し、複数の条件分岐、ネストされた可能性のあるループ（自己改善ループとリトライ）、人間による判断の組み込みを組み合わせた、実践的な多段ワークフローを構築する経験を積みます。

In [ ]:
# ▼▼▼▼▼▼▼▼▼▼ YOUR CODE HERE ▼▼▼▼▼▼▼▼▼▼
from typing import TypedDict, Annotated, Optional, List
import random
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# --- 状態定義 (State) ---
class IterativeApprovalState(TypedDict):
    messages: Annotated[list, add_messages]
    user_request: str
    current_proposal: Optional[str]
    self_eval_score: Optional[int] # 自己評価スコア (例: 1-5)
    improvement_attempts: int
    max_improvement_attempts: int
    human_approved: Optional[bool]
    human_feedback: Optional[str]
    error_message: Optional[str]
    task_retry_count: int
    max_task_retries: int

# --- ノード定義 (Nodes) ---
def generate_initial_proposal_node(state: IterativeApprovalState):
    req = state["user_request"]
    retry = state.get("task_retry_count", 0)
    print(f"generate_initial_proposal_node (Retry {retry}): Generating for '{req}'")
    if random.random() < 0.3 and retry < state["max_task_retries"]: # たまに失敗するダミー
        return {"error_message": "Proposal generation API failed", "task_retry_count": retry + 1}
    proposal = f"「{req}」の初期提案です。バージョン1。"
    return {"current_proposal": proposal, "improvement_attempts": 0, "error_message": None, "task_retry_count": 0, "messages":[AIMessage(content=f"初期提案: {proposal}")]}

def self_evaluate_proposal_node(state: IterativeApprovalState):
    proposal = state["current_proposal"]
    # ダミー評価: バージョンが上がるほど高スコア
    score = state.get("improvement_attempts", 0) + 1 
    print(f"self_evaluate_proposal_node: Proposal '{proposal}' scored {score}")
    return {"self_eval_score": score, "messages":[AIMessage(content=f"自己評価スコア: {score}")]}

def improve_proposal_node(state: IterativeApprovalState):
    old_proposal = state["current_proposal"]
    attempts = state.get("improvement_attempts", 0) + 1
    new_proposal = f"{old_proposal.split('。')[0]}。改善版バージョン{attempts+1}。"
    print(f"improve_proposal_node (Attempt {attempts}): Improved to '{new_proposal}'")
    return {"current_proposal": new_proposal, "improvement_attempts": attempts, "self_eval_score": None, "messages":[AIMessage(content=f"改善提案: {new_proposal}")]}

def human_final_approval_node(state: IterativeApprovalState):
    print(f"\n---人間による最終承認--- \n提案: {state['current_proposal']}")
    approved = input("承認しますか? (yes/no): ").lower() == "yes"
    feedback = input("フィードバック (任意): ") if not approved else None
    return {"human_approved": approved, "human_feedback": feedback, "messages":[HumanMessage(content=f"人間承認: {'承認' if approved else '拒否'}, FB: {feedback}")]}

def process_final_result_node(state: IterativeApprovalState):
    msg = f"最終結果: {'承認されました。' if state['human_approved'] else '差し戻されました。フィードバック: ' + str(state['human_feedback'])}"
    print(f"process_final_result_node: {msg}")
    return {"messages":[AIMessage(content=msg)]}

def task_error_handler_node(state: IterativeApprovalState):
    err = state.get("error_message", "不明なタスクエラー")
    print(f"task_error_handler_node: タスクエラー発生 - {err}")
    return {"messages":[AIMessage(content=f"タスクエラー: {err}")]}

# --- ルーター関数 ---
def route_after_generation(state: IterativeApprovalState):
    if state.get("error_message") and state.get("task_retry_count",0) < state.get("max_task_retries",1):
        return "retry_generation"
    elif state.get("error_message"):
        return "handle_generation_error"
    return "evaluate_self"

def route_after_self_eval(state: IterativeApprovalState):
    if state.get("self_eval_score", 0) < 3 and state.get("improvement_attempts", 0) < state.get("max_improvement_attempts", 1):
        return "improve_proposal"
    return "request_human_approval"

def route_after_human_approval(state: IterativeApprovalState):
    return "process_result" # 承認でも差し戻しでも同じ最終処理ノードへ

# --- グラフ構築 ---
workflow = StateGraph(IterativeApprovalState)
nodes = [generate_initial_proposal_node, self_evaluate_proposal_node, improve_proposal_node, 
           human_final_approval_node, process_final_result_node, task_error_handler_node]
node_names = ["generator", "self_evaluator", "improver", "human_approver", "result_processor", "task_error_handler"]
for name, node_func in zip(node_names, nodes):
    workflow.add_node(name, node_func)

workflow.set_entry_point("generator")
workflow.add_conditional_edges("generator", route_after_generation, 
                               {"retry_generation":"generator", "handle_generation_error":"task_error_handler", "evaluate_self":"self_evaluator"})
workflow.add_conditional_edges("self_evaluator", route_after_self_eval,
                               {"improve_proposal":"improver", "request_human_approval":"human_approver"})
workflow.add_edge("improver", "self_evaluator") # 改善後、再度自己評価へ
workflow.add_conditional_edges("human_approver", route_after_human_approval, {"process_result":"result_processor"})
workflow.add_edge("result_processor", END)
workflow.add_edge("task_error_handler", END)

graph = workflow.compile()

# --- 実行 (承認ケース) ---
print("--- 承認ケースのテスト --- (途中でyesと入力)")
inputs_approve = {
    "messages": [HumanMessage(content="新しいマーケティングスローガンを考えてください")],
    "user_request": "新しいマーケティングスローガン", 
    "max_improvement_attempts": 1, "max_task_retries": 1
}
for event in graph.stream(inputs_approve, {"recursion_limit": 20}): print(event)
# ▲▲▲▲▲▲▲▲▲▲ YOUR CODE HERE ▲▲▲▲▲▲▲▲▲▲

### 解答例

<details><summary>解答例を見る</summary>

``````python
from typing import TypedDict, Annotated, Optional, List
import random
import time
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class IterativeApprovalState(TypedDict):
    messages: Annotated[list, add_messages]
    user_request: str                 # ユーザーからの最初の指示
    current_proposal: Optional[str]   # 現在の提案内容
    self_eval_score: Optional[int]    # 自己評価スコア (例: 1-5、高いほど良い)
    improvement_attempts: int         # 自己改善の試行回数
    max_improvement_attempts: int     # 最大自己改善回数
    human_approved: Optional[bool]    # 人間による最終承認 (True/False)
    human_feedback: Optional[str]     # 人間からのフィードバック
    error_message: Optional[str]      # 直近のタスクエラーメッセージ
    task_retry_count: int            # タスク（提案生成など）のリトライ回数
    max_task_retries: int            # タスクの最大リトライ回数
    final_outcome: Optional[str]       # 最終的な結果メッセージ

# --- ノード定義 (Nodes) ---
def initialize_workflow_node(state: IterativeApprovalState):
    user_req = state["messages"][-1].content
    print(f"initialize_workflow_node: ユーザーリクエスト '{user_req}' でワークフロー開始。")
    return {
        "user_request": user_req,
        "current_proposal": None,
        "self_eval_score": None,
        "improvement_attempts": 0,
        "human_approved": None,
        "human_feedback": None,
        "error_message": None,
        "task_retry_count": 0,
        "final_outcome": None
    }

def generate_proposal_node(state: IterativeApprovalState):
    req = state["user_request"]
    retry = state.get("task_retry_count", 0)
    max_retries_for_task = state.get("max_task_retries", 1)
    print(f"generate_proposal_node (リトライ {retry+1}/{max_retries_for_task}): リクエスト '{req}' の提案を生成中...")
    
    # ダミーのAPI呼び出し失敗シミュレーション
    if retry < max_retries_for_task and random.random() < 0.4: # 40%の確率で初回リトライ可能時に失敗
        err_msg = "提案生成APIへの接続に一時的に失敗しました。"
        print(f"  -> 失敗: {err_msg}")
        # time.sleep(1)
        return {"error_message": err_msg, "task_retry_count": retry + 1, "messages":[AIMessage(content=f"提案生成試行 {retry+1} 失敗: {err_msg}")]}
    
    # 実際の提案生成 (LLM呼び出しなど)
    # proposal = llm.invoke(f"「{req}」に関する提案を作成してください。").content
    proposal = f"これが「{req}」に関する素晴らしい提案です。バージョン{state.get('improvement_attempts',0)+1}。内容は完璧です。"
    print(f"  -> 生成された提案: '{proposal}'")
    return {"current_proposal": proposal, "error_message": None, "task_retry_count": 0, "messages":[AIMessage(content=f"生成された提案 (v{state.get('improvement_attempts',0)+1}): {proposal}")]}

def self_evaluate_proposal_node(state: IterativeApprovalState):
    proposal = state.get("current_proposal", "(提案なし)")
    attempts = state.get("improvement_attempts", 0)
    print(f"self_evaluate_proposal_node: 提案 '{proposal[:30]}...' を自己評価中 (改善試行: {attempts})...")
    
    # ダミー評価ロジック: 試行回数が少ないうちは低いスコア
    score = 2 + attempts * 2 # 試行ごとにスコアが2ずつ上がる (最大改善回数による)
    if score > 5: score = 5
    print(f"  -> 自己評価スコア: {score}/5")
    return {"self_eval_score": score, "messages":[AIMessage(content=f"自己評価スコア: {score}/5")]}

def improve_proposal_node(state: IterativeApprovalState):
    old_proposal = state.get("current_proposal", "(元の提案なし)")
    attempts = state.get("improvement_attempts", 0) + 1
    print(f"improve_proposal_node (改善試行 {attempts}/{state['max_improvement_attempts']}): 提案 '{old_proposal[:30]}...' を改善中...")
    
    # 実際の改善処理 (LLM呼び出しなど)
    # new_proposal = llm.invoke(f"以下の提案を改善してください: {old_proposal}").content
    new_proposal = f"{old_proposal.split('。')[0]}。さらに洗練されたバージョン{attempts+1}です。今度こそ完璧。"
    print(f"  -> 改善された提案: '{new_proposal}'")
    return {"current_proposal": new_proposal, "improvement_attempts": attempts, "self_eval_score": None, "messages":[AIMessage(content=f"改善された提案 (v{attempts+1}): {new_proposal}")]}

def human_final_approval_node(state: IterativeApprovalState):
    proposal_to_approve = state.get("current_proposal", "(最終提案なし)")
    print(f"\n--- 人間による最終承認ステップ --- ")
    print(f"ユーザーリクエスト: {state['user_request']}")
    print(f"最終提案:\n{'-'*30}\n{proposal_to_approve}\n{'-'*30}")
    
    approval_input = ""
    while approval_input not in ["yes", "no"]:
        approval_input = input("この最終提案を承認しますか？ (yes/no): ").strip().lower()
    
    approved = (approval_input == "yes")
    feedback_text = None
    human_action_message = ""
    
    if approved:
        print("  -> 承認されました。")
        human_action_message = "人間によって最終承認されました。"
    else:
        print("  -> 差し戻し (拒否) となりました。")
        feedback_text = input("差し戻しの理由やフィードバックを入力してください (任意): ").strip()
        if not feedback_text: feedback_text = "(具体的フィードバックなし)"
        print(f"  -> 受け取ったフィードバック: '{feedback_text}'")
        human_action_message = f"人間によって差し戻されました。フィードバック: {feedback_text}"
        
    return {"human_approved": approved, "human_feedback": feedback_text, "messages":[HumanMessage(content=human_action_message)]}

def process_final_outcome_node(state: IterativeApprovalState):
    if state.get("human_approved") is True:
        outcome_message = f"最終提案「{state.get('current_proposal', '(不明な提案)')[:30]}...」は承認され、ワークフローは正常に完了しました。"
    else:
        outcome_message = f"最終提案は差し戻されました。フィードバック: 「{state.get('human_feedback', '(フィードバックなし)')}」。ワークフローはここで終了します。"
    print(f"process_final_outcome_node: {outcome_message}")
    return {"final_outcome": outcome_message, "messages":[AIMessage(content=outcome_message)]}

def task_error_handler_node(state: IterativeApprovalState):
    err = state.get("error_message", "不明なタスクエラー")
    outcome_message = f"タスク処理中に解決不能なエラーが発生しました: {err}。ワークフローを終了します。"
    print(f"task_error_handler_node: {outcome_message}")
    return {"final_outcome": outcome_message, "messages":[AIMessage(content=outcome_message)]}

# --- ルーター関数 ---
def route_after_proposal_generation(state: IterativeApprovalState):
    print(f"route_after_proposal_generation: エラー '{state.get('error_message')}', リトライ回数 {state.get('task_retry_count',0)}/{state.get('max_task_retries',1)}")
    if state.get("error_message") and state.get("task_retry_count", 0) < state.get("max_task_retries", 1):
        print("  -> 提案生成リトライ (generatorへ)")
        return "retry_proposal_generation"
    elif state.get("error_message"):
        print("  -> 提案生成エラー上限。エラーハンドラ (task_error_handlerへ)")
        return "handle_generation_failure"
    print("  -> 提案生成成功。自己評価 (self_evaluatorへ)")
    return "proceed_to_self_evaluation"

def route_after_self_evaluation(state: IterativeApprovalState):
    score = state.get("self_eval_score", 0)
    attempts = state.get("improvement_attempts", 0)
    max_attempts = state.get("max_improvement_attempts", 1)
    print(f"route_after_self_evaluation: 自己評価スコア {score}, 改善試行 {attempts}/{max_attempts}")
    if score < 4 and attempts < max_attempts: # スコア4未満、かつ改善上限未満なら改善へ
        print("  -> スコア不十分。改善 (improverへ)")
        return "needs_improvement"
    print("  -> スコア十分または改善上限。人間による承認 (human_approverへ)")
    return "ready_for_human_approval"

def route_after_human_approval(state: IterativeApprovalState):
    # このデモでは、承認でも差し戻しでも同じ最終結果処理ノードへ行く
    # 実際には、差し戻されたら再度改善ループに戻るなどの分岐も考えられる
    print(f"route_after_human_approval: 人間の判断 -> {'承認' if state.get('human_approved') else '差し戻し'}。最終結果処理 (result_processorへ)")
    return "proceed_to_final_outcome" 

# --- グラフ構築 ---
workflow = StateGraph(IterativeApprovalState)

workflow.add_node("initializer", initialize_workflow_node)
workflow.add_node("generator", generate_proposal_node)
workflow.add_node("self_evaluator", self_evaluate_proposal_node)
workflow.add_node("improver", improve_proposal_node)
workflow.add_node("human_approver", human_final_approval_node)
workflow.add_node("result_processor", process_final_outcome_node)
workflow.add_node("task_error_handler", task_error_handler_node)

workflow.set_entry_point("initializer")
workflow.add_edge("initializer", "generator")

workflow.add_conditional_edges(
    "generator", 
    route_after_proposal_generation, 
    {
        "retry_proposal_generation": "generator", 
        "handle_generation_failure": "task_error_handler", 
        "proceed_to_self_evaluation": "self_evaluator"
    }
)
workflow.add_conditional_edges(
    "self_evaluator", 
    route_after_self_evaluation,
    {
        "needs_improvement": "improver", 
        "ready_for_human_approval": "human_approver"
    }
)
workflow.add_edge("improver", "self_evaluator") # 改善後、再度自己評価へループバック
workflow.add_conditional_edges(
    "human_approver", 
    route_after_human_approval, 
    {"proceed_to_final_outcome": "result_processor"}
)

workflow.add_edge("result_processor", END)
workflow.add_edge("task_error_handler", END)

graph = workflow.compile()

# --- グラフの可視化 ---
try:
    display(Image(graph.get_graph().draw_png()))
except Exception as e:
    print(f"グラフの可視化に失敗: {e}. Graphvizがインストールされているか確認してください。")

# --- 実行 (承認されるケースをシミュレート) ---
print("\n--- 第2章まとめテスト (ユーザーが 'yes' と入力して承認するケース) ---")
inputs_approve_case = {
    "messages": [HumanMessage(content="新しい環境保護キャンペーンのスローガンを提案してください")],
    "max_improvement_attempts": 2, # 自己改善は最大2回まで
    "max_task_retries": 1         # 提案生成タスクのリトライは1回まで
}
print("\nストリーム出力 (承認ケース):")
for event in graph.stream(inputs_approve_case, {"recursion_limit": 25}): # ループがあるのでrecursion_limitに注意
    print(event)

print("\n最終状態の確認 (承認ケース):")
final_state_approve = graph.invoke(inputs_approve_case, {"recursion_limit": 25})
print(f"  最終的な結果: {final_state_approve.get('final_outcome')}")
print(f"  最終提案: {final_state_approve.get('current_proposal')}")
print(f"  人間による承認: {final_state_approve.get('human_approved')}")

# --- 実行 (差し戻されるケースをシミュレート) ---
print("\n--- 第2章まとめテスト (ユーザーが 'no' と入力して差し戻すケース) ---")
inputs_reject_case = {
    "messages": [HumanMessage(content="子供向けの新しい教育アプリのアイデアを出してください")],
    "max_improvement_attempts": 1, 
    "max_task_retries": 1
}
print("\nストリーム出力 (差し戻しケース):")
# input() をモックするか、手動で入力する必要がある。
# この自動テストでは、手動入力の代わりに固定値を想定して進めるのは難しい。
# ここでは実行ログで人間の入力が求められることを確認するに留める。
# 実際のテストでは、 `unittest.mock.patch('builtins.input', side_effect=['no', 'もっとインタラクティブに！'])` のようにする
print("  (実行時、'no'と入力し、フィードバックを求められたら何か入力してください)")
for event in graph.stream(inputs_reject_case, {"recursion_limit": 25}):
    print(event)

print("\n最終状態の確認 (差し戻しケース - 手動入力結果に依存):")
# final_state_reject = graph.invoke(inputs_reject_case, {"recursion_limit": 25})
# print(f"  最終的な結果: {final_state_reject.get('final_outcome')}")
# print(f"  人間による承認: {final_state_reject.get('human_approved')}")
# print(f"  人間からのFB: {final_state_reject.get('human_feedback')}")
print("  (上記は手動入力の結果に依存するため、コメントアウトしています)")
``````
</details>